In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import string
import json
import re
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.metrics import accuracy_score
####
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)
italian_ingredients = list(set(italian_df.ingredients))

mexican_df = DataFrame(clean_mexican_cuisine_data)
mexican_ingredients = list(set(mexican_df.ingredients))


In [5]:
#fixing data for CountVectorizer (italian_df)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)
    

In [6]:
#fixing data for CountVectorizer (mexican_df)

mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    


In [7]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()

it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')
print(it_df)

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()

mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')
print(mex_df)

       abbamele  accent  acinipepe  acornsquash  activedryyeast  adobosauce  \
4             0       0          0            0               0           0   
14            0       0          0            0               0           0   
20            0       0          0            0               0           0   
56            0       0          0            0               0           0   
62            0       0          0            0               0           0   
...         ...     ...        ...          ...             ...         ...   
49679         0       0          0            0               0           0   
49681         0       0          0            0               0           0   
49689         0       0          0            0               0           0   
49697         0       0          0            0               0           0   
49708         0       0          0            0               0           0   

       adoboseasoning  agavenectar  agedbalsamicvin

In [8]:
#putting mexican and italian in one DataFrame

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

In [9]:
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

(14276, 3976)


In [10]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [11]:
# Multiple Logistic Regression 

lr_m = linear_model.LogisticRegression()
#lr_m.fit(X, Y)

In [12]:
'''
print("classifier score:",lr_m.score(X_test, Y_test))
Y_pred = lr_m.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average = None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)
'''

'\nprint("classifier score:",lr_m.score(X_test, Y_test))\nY_pred = lr_m.predict(X_test)\nprint("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))\nprint("\nConfusion matrix:")\nprint(metrics.confusion_matrix(Y_test, Y_pred))\nprint("\nPrecision Score per class:")\nprint(metrics.precision_score(Y_test, Y_pred, average = None))\nprint("\nAverage Precision Score:")\nprint(metrics.precision_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nRecall Score per class:")\nprint(metrics.recall_score(Y_test, Y_pred, average = None))\nprint("\nAverage Recall Score:")\nprint(metrics.recall_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nF1-score Score per class:")\nprint(metrics.f1_score(Y_test, Y_pred, average = None))\nprint("\nAverage F1 Score:")\nprint(metrics.f1_score(Y_test, Y_pred, average = \'weighted\'))\n\nreport = metrics.classification_report(Y_test,Y_pred)\nprint(report)\n'

In [13]:
scores = model_selection.cross_val_score(lr_m, X, Y, scoring = 'f1_weighted', cv = 5)
print('scores:', scores)
print('mean scores:', scores.mean())

scores: [0.96809087 0.97195581 0.96879183 0.96668815 0.96880957]
mean scores: 0.9688672451742961


In [14]:
# Support Vector Machines

svm_m = svm.SVC()
#svm_m.fit(X, Y)

In [15]:
'''
print("classifier score:",svm_m.score(X_test, Y_test))
Y_pred = svm_m.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average= None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)
'''

'\nprint("classifier score:",svm_m.score(X_test, Y_test))\nY_pred = svm_m.predict(X_test)\nprint("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))\nprint("\nConfusion matrix:")\nprint(metrics.confusion_matrix(Y_test, Y_pred))\nprint("\nPrecision Score per class:")\nprint(metrics.precision_score(Y_test, Y_pred, average = None))\nprint("\nAverage Precision Score:")\nprint(metrics.precision_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nRecall Score per class:")\nprint(metrics.recall_score(Y_test, Y_pred, average= None))\nprint("\nAverage Recall Score:")\nprint(metrics.recall_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nF1-score Score per class:")\nprint(metrics.f1_score(Y_test, Y_pred, average = None))\nprint("\nAverage F1 Score:")\nprint(metrics.f1_score(Y_test, Y_pred, average = \'weighted\'))\n\nreport = metrics.classification_report(Y_test,Y_pred)\nprint(report)\n'

In [16]:
#Decision Trees

dtree = tree.DecisionTreeClassifier()
#dtree.fit(X, Y)

In [17]:
'''
print("classifier score:",dtree.score(X_test, Y_test))
Y_pred = dtree.predict(X_test)
Y_prob = dtree.predict_proba(X_test)
print('Y probability:', Y_prob)

print("classifier predictions:", Y_pred)
print("ground truth labels   :", Y_test)

print("accuracy:", metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average=None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average='weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average=None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average='weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average=None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average='weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)
'''

'\nprint("classifier score:",dtree.score(X_test, Y_test))\nY_pred = dtree.predict(X_test)\nY_prob = dtree.predict_proba(X_test)\nprint(\'Y probability:\', Y_prob)\n\nprint("classifier predictions:", Y_pred)\nprint("ground truth labels   :", Y_test)\n\nprint("accuracy:", metrics.accuracy_score(Y_test, Y_pred))\nprint("\nConfusion matrix:")\nprint(metrics.confusion_matrix(Y_test, Y_pred))\nprint("\nPrecision Score per class:")\nprint(metrics.precision_score(Y_test, Y_pred, average=None))\nprint("\nAverage Precision Score:")\nprint(metrics.precision_score(Y_test, Y_pred, average=\'weighted\'))\nprint("\nRecall Score per class:")\nprint(metrics.recall_score(Y_test, Y_pred, average=None))\nprint("\nAverage Recall Score:")\nprint(metrics.recall_score(Y_test, Y_pred, average=\'weighted\'))\nprint("\nF1-score Score per class:")\nprint(metrics.f1_score(Y_test, Y_pred, average=None))\nprint("\nAverage F1 Score:")\nprint(metrics.f1_score(Y_test, Y_pred, average=\'weighted\'))\n\nreport = metrics.

In [18]:
#k-NN

knn = KNeighborsClassifier(n_neighbors = 3)
#knn.fit(X,Y)

In [19]:
'''
print("classifier score:", knn.score(X_test, Y_test))
Y_pred = knn.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average = None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print('Report:', report)
'''

'\nprint("classifier score:", knn.score(X_test, Y_test))\nY_pred = knn.predict(X_test)\nprint("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))\nprint("\nConfusion matrix:")\nprint(metrics.confusion_matrix(Y_test, Y_pred))\nprint("\nPrecision Score per class:")\nprint(metrics.precision_score(Y_test, Y_pred, average = None))\nprint("\nAverage Precision Score:")\nprint(metrics.precision_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nRecall Score per class:")\nprint(metrics.recall_score(Y_test, Y_pred, average = None))\nprint("\nAverage Recall Score:")\nprint(metrics.recall_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nF1-score Score per class:")\nprint(metrics.f1_score(Y_test, Y_pred, average = None))\nprint("\nAverage F1 Score:")\nprint(metrics.f1_score(Y_test, Y_pred, average = \'weighted\'))\n\nreport = metrics.classification_report(Y_test,Y_pred)\nprint(\'Report:\', report)\n'

In [20]:
# Naive Bayes

gnb = BernoulliNB(binarize = None)
#gnb.fit(X, Y)

In [21]:
'''
print("classifier score:", gnb.score(X_test, Y_test))
Y_pred = gnb.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average= None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)
'''

'\nprint("classifier score:", gnb.score(X_test, Y_test))\nY_pred = gnb.predict(X_test)\nprint("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))\nprint("\nConfusion matrix:")\nprint(metrics.confusion_matrix(Y_test, Y_pred))\nprint("\nPrecision Score per class:")\nprint(metrics.precision_score(Y_test, Y_pred, average = None))\nprint("\nAverage Precision Score:")\nprint(metrics.precision_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nRecall Score per class:")\nprint(metrics.recall_score(Y_test, Y_pred, average= None))\nprint("\nAverage Recall Score:")\nprint(metrics.recall_score(Y_test, Y_pred, average = \'weighted\'))\nprint("\nF1-score Score per class:")\nprint(metrics.f1_score(Y_test, Y_pred, average = None))\nprint("\nAverage F1 Score:")\nprint(metrics.f1_score(Y_test, Y_pred, average = \'weighted\'))\n\nreport = metrics.classification_report(Y_test,Y_pred)\nprint(report)\n'

In [23]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())


y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, y_pred)
print(conf_matrix)
#Y_pred = lr_m.predict(X_test)

****Logistic Regression mean scores****
scores: {'fit_time': array([2.37456536, 2.2973938 , 2.26086092, 2.15168166, 2.19990587]), 'score_time': array([0.02655697, 0.0269928 , 0.02665615, 0.02711749, 0.02799129]), 'test_accuracy': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662]), 'test_precision_weighted': array([0.96837424, 0.9720495 , 0.96895965, 0.96685339, 0.96884661]), 'test_recall_weighted': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662]), 'test_f1_weighted': array([0.96809087, 0.97195581, 0.96879183, 0.96668815, 0.96880957])}

mean score test accuracy: 0.9688989045705847

mean score test precision weighted: 0.9690166784300608

mean score test recall weighted: 0.9688989045705847

mean score test f1-measure weighted: 0.9688672451742961


In [ ]:
#sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

In [ ]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****d-tree mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

In [ ]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

In [ ]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())